# 감정 분석

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### 감정분석 특화 사전 학습 모델로 학습하기
#### hun3359/klue-bert-base-sentiment


In [ ]:
#!pip install kss

In [ ]:
#!pip install --upgrade transformers

In [ ]:
# 세션 다시 시작

In [ ]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from kss import split_sentences
import random
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score


In [ ]:
# 감정 컬럼 정의
emotion_columns = [
    '감사한','그저 그런','기쁜','당황한','만족스러운','분노한','불안한',
    '스트레스 받는','슬픈','외로운','우울한','자신하는','편안한','흥분한'
]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/diary_sentiment_analysis/train_diary_with_emotions.csv")
df

,category,title,text,exist_emotions
0,비출판물 > 일기,5학년을 마치며....,5학년을 마치며.... ...,"['기쁜', '감사한', '슬픈']"
1,비출판물 > 일기,<name1>의 깜짝 등장,2018/1/20 토요일 날씨 오늘은 <name2>이의 유치원친구들이 놀러왔다.일등...,"['기쁜', '만족스러운']"
2,비출판물 > 일기,<name2>집에 초대,<name1>누나랑 같이 <name2>집에같다 <name2> 놀이터에서 물이나와 서...,"['기쁜', '만족스러운']"
3,비출판물 > 일기,Happy,다언어 합창제를 했다. 우리 라보 노래 제목은 Happy였다. 그런데 내 짝꿍이 나...,"['기쁜', '만족스러운', '감사한']"
4,비출판물 > 일기,TV,앞으로 꼭 발명되어야 하든 발명품은 똑똑상자 TV입니다. 왜냐하면 제가 TV를 좋아...,"['기쁜', '만족스러운']"
...,...,...,...,...
4175,비출판물 > 일기,힘들다,요즈음 엄마가 회사 일 때문에 바쁘셔서 밥을 못 챙겨 주신다. 나는 밥과 반찬을 잘...,"['슬픈', '흥분한', '불안한']"
4176,비출판물 > 일기,힘들면서 좋았던 날,<힘들면서 좋았던 날> 오늘 아침 8시에 일어났다. 조금 피곤했지만 또 잠들기는 싫...,['그저 그런']
4177,비출판물 > 일기,힘들었던 날.,오늘은 감정적으로 매우 힘든 날이었다. 우울하고 지치고... 나갈 곳은 없고 하고싶...,"['우울한', '슬픈', '외로운']"
4178,비출판물 > 일기,힘은 솟아나지 않는다.,힘은 솟아나지 않는다. 힘이란 연속되면서 이어진다. 관성과 같다. 나는 언제나 솟구...,"['우울한', '슬픈', '외로운']"


In [ ]:
print(df['exist_emotions'].head(10))
print(df['exist_emotions'].apply(type).value_counts())

0        ['기쁜', '감사한', '슬픈']
1            ['기쁜', '만족스러운']
2            ['기쁜', '만족스러운']
3     ['기쁜', '만족스러운', '감사한']
4            ['기쁜', '만족스러운']
5    ['자신하는', '기쁜', '만족스러운']
6              ['슬픈', '분노한']
7     ['기쁜', '만족스러운', '감사한']
8            ['기쁜', '만족스러운']
9     ['기쁜', '불안한', '만족스러운']
Name: exist_emotions, dtype: object
exist_emotions
<class 'str'>    4180
Name: count, dtype: int64


In [ ]:

# 1. 문자열을 리스트로 안전하게 변환
def safe_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return []
    elif isinstance(val, list):
        return val
    else:
        return []

df['exist_emotions'] = df['exist_emotions'].apply(safe_literal_eval)

# 2. 데이터 전체에서 고유 감정 라벨 추출
all_emotions = sorted(
    set(
        emotion.strip()
        for emotions in df['exist_emotions']
        for emotion in emotions
        if isinstance(emotion, str) and len(emotion.strip()) > 1
    )
)
print("고유 감정 라벨:", all_emotions)

# 3. 멀티-핫 인코딩
mlb = MultiLabelBinarizer(classes=all_emotions)
multi_hot = mlb.fit_transform(df['exist_emotions'])
df_multi_hot = pd.DataFrame(multi_hot, columns=mlb.classes_)

# 4. 결과 확인
print(df_multi_hot.head())


고유 감정 라벨: ['감사한', '그저 그런', '기쁜', '당황한', '만족스러운', '분노한', '불안한', '스트레스 받는', '슬픈', '외로운', '우울한', '자신하는', '편안한', '흥분한']
   감사한  그저 그런  기쁜  당황한  만족스러운  분노한  불안한  스트레스 받는  슬픈  외로운  우울한  자신하는  편안한  흥분한
0    1      0   1    0      0    0    0        0   1    0    0     0    0    0
1    0      0   1    0      1    0    0        0   0    0    0     0    0    0
2    0      0   1    0      1    0    0        0   0    0    0     0    0    0
3    1      0   1    0      1    0    0        0   0    0    0     0    0    0
4    0      0   1    0      1    0    0        0   0    0    0     0    0    0


In [ ]:
df_final = pd.concat([df, df_multi_hot], axis=1)
df_final

,category,title,text,exist_emotions,감사한,그저 그런,기쁜,당황한,만족스러운,분노한,불안한,스트레스 받는,슬픈,외로운,우울한,자신하는,편안한,흥분한
0,비출판물 > 일기,5학년을 마치며....,5학년을 마치며.... ...,"[기쁜, 감사한, 슬픈]",1,0,1,0,0,0,0,0,1,0,0,0,0,0
1,비출판물 > 일기,<name1>의 깜짝 등장,2018/1/20 토요일 날씨 오늘은 <name2>이의 유치원친구들이 놀러왔다.일등...,"[기쁜, 만족스러운]",0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,비출판물 > 일기,<name2>집에 초대,<name1>누나랑 같이 <name2>집에같다 <name2> 놀이터에서 물이나와 서...,"[기쁜, 만족스러운]",0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,비출판물 > 일기,Happy,다언어 합창제를 했다. 우리 라보 노래 제목은 Happy였다. 그런데 내 짝꿍이 나...,"[기쁜, 만족스러운, 감사한]",1,0,1,0,1,0,0,0,0,0,0,0,0,0
4,비출판물 > 일기,TV,앞으로 꼭 발명되어야 하든 발명품은 똑똑상자 TV입니다. 왜냐하면 제가 TV를 좋아...,"[기쁜, 만족스러운]",0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,비출판물 > 일기,힘들다,요즈음 엄마가 회사 일 때문에 바쁘셔서 밥을 못 챙겨 주신다. 나는 밥과 반찬을 잘...,"[슬픈, 흥분한, 불안한]",0,0,0,0,0,0,1,0,1,0,0,0,0,1
4176,비출판물 > 일기,힘들면서 좋았던 날,<힘들면서 좋았던 날> 오늘 아침 8시에 일어났다. 조금 피곤했지만 또 잠들기는 싫...,[그저 그런],0,1,0,0,0,0,0,0,0,0,0,0,0,0
4177,비출판물 > 일기,힘들었던 날.,오늘은 감정적으로 매우 힘든 날이었다. 우울하고 지치고... 나갈 곳은 없고 하고싶...,"[우울한, 슬픈, 외로운]",0,0,0,0,0,0,0,0,1,1,1,0,0,0
4178,비출판물 > 일기,힘은 솟아나지 않는다.,힘은 솟아나지 않는다. 힘이란 연속되면서 이어진다. 관성과 같다. 나는 언제나 솟구...,"[우울한, 슬픈, 외로운]",0,0,0,0,0,0,0,0,1,1,1,0,0,0


In [ ]:
# 클래스 가중치(pos_weight) 계산
pos_counts = df_multi_hot.sum(axis=0).values
neg_counts = len(df_multi_hot) - pos_counts
pos_weight = torch.tensor(neg_counts / pos_counts, dtype=torch.float)

In [ ]:
# 토크나이저 불러오기
MODEL_NAME = "hun3359/klue-bert-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# chunk 함수 정의

def chunk_text_with_kss(text, tokenizer, max_tokens=512):
    sentences = split_sentences(str(text))
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        encoded = tokenizer.encode(sentence, add_special_tokens=True)
        tokenized_len = len(encoded)
        if tokenized_len > max_tokens:
            words = sentence.split()
            chunk = []
            chunk_len = 0
            for word in words:
                word_encoded = tokenizer.encode(word, add_special_tokens=False)
                word_len = len(word_encoded)
                if chunk_len + word_len > max_tokens:
                    chunks.append(' '.join(chunk))
                    chunk = []
                    chunk_len = 0
                chunk.append(word)
                chunk_len += word_len
            if chunk:
                chunks.append(' '.join(chunk))
            current_length = 0
            current_chunk = []
            continue
        if current_length + tokenized_len <= max_tokens:
            current_chunk.append(sentence)
            current_length += tokenized_len
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_length = tokenized_len
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks


In [ ]:
# chunk별 학습 데이터 생성

train_texts = []
train_labels = []
diary_ids = []

for idx, row in tqdm(df_final.iterrows(), total=len(df_final)):
    text = row['text']
    labels = row[emotion_columns].values.astype(int)
    chunks = chunk_text_with_kss(text, tokenizer, max_tokens=512)
    for chunk in chunks:
        train_texts.append(chunk)
        train_labels.append(labels)
        diary_ids.append(idx)


  0%|          | 0/4180 [00:00<?, ?it/s]WARNING:root:Oh! You have mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 4180/4180 [01:52<00:00, 37.27it/s]


In [ ]:
print("총 학습 데이터(Chunk) 개수:", len(train_texts))
print(len(train_labels))
print(len(diary_ids))

총 학습 데이터(Chunk) 개수: 5434
5434
5434


In [ ]:
# 학습/검증 데이터 분할

X_train, X_val, y_train, y_val, train_diary_ids, val_diary_ids = train_test_split(
    train_texts, train_labels, diary_ids, test_size=0.2, random_state=1234
)


In [ ]:
# 토크나이즈

def tokenize_function(texts):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

train_encodings = tokenize_function(X_train)
val_encodings = tokenize_function(X_val)


In [ ]:

# PyTorch Dataset 정의

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels, diary_ids):
        self.encodings = encodings
        self.labels = labels
        self.diary_ids = diary_ids

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        item['diary_id'] = self.diary_ids[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, y_train, train_diary_ids)
val_dataset = EmotionDataset(val_encodings, y_val, val_diary_ids)

In [ ]:

# set_seed 함수 정의

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# seed 고정 (모델 초기화 전)
set_seed(1234)



# 커스텀 Trainer 정의 (여기서 pos_weight 사용)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(emotion_columns),
    problem_type="multi_label_classification",
    ignore_mismatched_sizes=True
)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/diary_sentiment_analysis/klue_bert_base_sentiment_results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_dir='/content/drive/MyDrive/diary_sentiment_analysis/klue_bert_base_sentiment_logs',
    logging_steps=100,
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

config.json:   0%|          | 0.00/3.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hun3359/klue-bert-base-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([14]) in the model instantiated
- classifier.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([14, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 모델 학습

trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=408, training_loss=0.6496743291032081, metrics={'train_runtime': 1190.8332, 'train_samples_per_second': 10.951, 'train_steps_per_second': 0.343, 'total_flos': 3431600964642816.0, 'train_loss': 0.6496743291032081, 'epoch': 3.0})

In [ ]:
# 예측 및 일기 단위 결과 집계

# 검증 데이터셋: X_val (chunk 텍스트), val_diary_ids (각 chunk의 일기 인덱스)

val_df = pd.DataFrame({'text': X_val, 'diary_id': val_diary_ids})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = trainer.model
model.to(device)

results = []
threshold = 0.4
for diary_id, group in val_df.groupby('diary_id'):
    all_probs = []
    for chunk in group['text']:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True, max_length=512)
        # 모든 입력 텐서를 GPU로 이동
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]
            all_probs.append(probs)
    mean_probs = np.mean(all_probs, axis=0)
    predicted_labels = (mean_probs >= threshold).astype(int)
    predicted_emotions = [emotion_columns[i] for i, p in enumerate(predicted_labels) if p == 1]
    results.append({
        'diary_id': diary_id,
        'predicted_emotions': predicted_emotions,
        'probs': mean_probs
    })

In [ ]:
#예측 결과와 실제 라벨로 평가 지표 계산

def compute_metrics(preds, labels):
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')
    precision_micro = precision_score(labels, preds, average='micro')
    recall_micro = recall_score(labels, preds, average='micro')
    hamming = hamming_loss(labels, preds)
    exact_match = accuracy_score(labels, preds)
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'hamming_loss': hamming,
        'exact_match_ratio': exact_match
    }

# 예측값과 실제값 준비
preds_list = []
labels_list = []
for r in results:
    preds_list.append(np.array([1 if e in r['predicted_emotions'] else 0 for e in emotion_columns]))
    true_labels = df_final.loc[r['diary_id'], emotion_columns].values.astype(int)
    labels_list.append(true_labels)

preds = np.array(preds_list)
labels = np.array(labels_list)

metrics = compute_metrics(preds, labels)

def print_metrics(metrics):
    print("Evaluation Metrics:")
    print(f"F1 Score (Micro): {metrics['f1_micro']:.4f}")
    print(f"F1 Score (Macro): {metrics['f1_macro']:.4f}")
    print(f"Precision (Micro): {metrics['precision_micro']:.4f}")
    print(f"Recall (Micro): {metrics['recall_micro']:.4f}")
    print(f"Hamming Loss: {metrics['hamming_loss']:.4f}")
    print(f"Exact Match Ratio: {metrics['exact_match_ratio']:.4f}")

print_metrics(metrics)


Evaluation Metrics:
F1 Score (Micro): 0.6264
F1 Score (Macro): 0.5568
Precision (Micro): 0.4934
Recall (Micro): 0.8575
Hamming Loss: 0.1962
Exact Match Ratio: 0.0584
